In [5]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
sys.path.append("../src")
import main

In [19]:
def find_best_param(radius=200, time = '2022-11-01 13:00:00+00:00'):

    temp =  pd.read_csv('/Users/lisawink/Documents/paper1/data/gap_filled_data_ta_rh.csv')
    temp['datetime_UTC']=pd.to_datetime(temp['datetime_UTC'])
    temp = temp[temp['variable']=='Ta_deg_C']
    temp['datetime_UTC'] = temp['datetime_UTC'].astype(str)
    temp = temp.pivot(index='station_id', columns='datetime_UTC', values='value')

    params = gpd.read_parquet(f'/Users/lisawink/Documents/paper1/data/processed_data/processed_station_params_{radius}.parquet')
    params = params.set_index('station_id')
    to_remove = ['station_no','station_name','station_long_name','station_type','mounting_structure','dominant_land_use','local_climate_zone','urban_atlas_class','urban_atlas_code','geometry','SVF_3D']
    params = params.drop(to_remove, axis=1)
    params = params.drop('FREBNE',axis=0)
    vars = params.columns

    stats_dict = {}

    for var in vars:
        params_v = params[[var]]

        params_t = params_v.merge(temp[time], left_on='station_id', right_on='station_id',how='inner')
        stats = main.calculate_statistics(params_t, time)
        stats.index = stats['Parameter']
        stats['Time'] = time
        stats_dict[var] = stats

    pd.concat(stats_dict).to_csv(f'/Users/lisawink/Documents/paper1/data/processed_data/best_param/{time,radius}.csv')
    return pd.concat(stats_dict)


In [20]:
params = find_best_param(radius=200, time='2022-11-01 13:00:00+00:00')

/Users/lisawink/Documents/freiburg/src/main.py:563: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_corr, pearson_pval = pearsonr(x, y)
/Users/lisawink/Documents/freiburg/src/main.py:566: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_corr, spearman_pval = spearmanr(x, y)
/Users/lisawink/Documents/freiburg/src/main.py:563: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_corr, pearson_pval = pearsonr(x, y)
/Users/lisawink/Documents/freiburg/src/main.py:566: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  spearman_corr, spearman_pval = spearmanr(x, y)
/Users/lisawink/Documents/freiburg/src/main.py:563: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearson_corr, pearson_pval = pearsonr(x, y)
/Users/lisawink/Documents/freiburg

In [21]:
params

,,Parameter,Pearson Correlation,Pearson p-value,Spearman Correlation,Spearman p-value,Mutual Information,Time
,Parameter,,,,,,,
station_lat,station_lat,station_lat,0.618569,3.486232e-05,0.538003,0.000494,0.236746,2022-11-01 13:00:00+00:00
station_lon,station_lon,station_lon,-0.311946,5.657107e-02,-0.386211,0.016621,0.043389,2022-11-01 13:00:00+00:00
station_elevation,station_elevation,station_elevation,-0.731376,1.831570e-07,-0.636916,0.000017,0.535261,2022-11-01 13:00:00+00:00
sky_view_factor,sky_view_factor,sky_view_factor,0.311590,5.686909e-02,0.448336,0.004758,0.077792,2022-11-01 13:00:00+00:00
station_elevation_diff,station_elevation_diff,station_elevation_diff,-0.731376,1.831570e-07,-0.636916,0.000017,0.533747,2022-11-01 13:00:00+00:00
...,...,...,...,...,...,...,...,...
SVF_3D_per75,SVF_3D_per75,SVF_3D_per75,0.142443,3.935995e-01,0.220444,0.183538,0.091987,2022-11-01 13:00:00+00:00
SVF_3D_IQR,SVF_3D_IQR,SVF_3D_IQR,0.009357,9.555352e-01,-0.121346,0.468004,0.000000,2022-11-01 13:00:00+00:00
SVF_3D_min,SVF_3D_min,SVF_3D_min,-0.209463,2.068993e-01,0.129364,0.438885,0.038684,2022-11-01 13:00:00+00:00


In [22]:
params.sort_values(by='Mutual Information', ascending=False)

,,Parameter,Pearson Correlation,Pearson p-value,Spearman Correlation,Spearman p-value,Mutual Information,Time
,Parameter,,,,,,,
station_elevation,station_elevation,station_elevation,-0.731376,1.831570e-07,-0.636916,0.000017,0.535261,2022-11-01 13:00:00+00:00
station_elevation_diff,station_elevation_diff,station_elevation_diff,-0.731376,1.831570e-07,-0.636916,0.000017,0.533747,2022-11-01 13:00:00+00:00
BuSpi_3D_median,BuSpi_3D_median,BuSpi_3D_median,-0.432234,9.512856e-03,-0.375963,0.026026,0.415478,2022-11-01 13:00:00+00:00
BuProx_3D_median,BuProx_3D_median,BuProx_3D_median,-0.432560,9.452958e-03,-0.374562,0.026632,0.401900,2022-11-01 13:00:00+00:00
BuProx_3D_per75,BuProx_3D_per75,BuProx_3D_per75,-0.372510,2.754140e-02,-0.343045,0.043647,0.349778,2022-11-01 13:00:00+00:00
...,...,...,...,...,...,...,...,...
BuRan_sum,BuRan_sum,BuRan_sum,-0.090609,5.937997e-01,-0.216492,0.198096,0.000000,2022-11-01 13:00:00+00:00
BuMVE_3D_median,BuMVE_3D_median,BuMVE_3D_median,NaN,NaN,NaN,NaN,0.000000,2022-11-01 13:00:00+00:00
BumVE_3D_skew,BumVE_3D_skew,BumVE_3D_skew,NaN,NaN,NaN,NaN,NaN,2022-11-01 13:00:00+00:00
